In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import numpy as np
import xgboost as xgb

In [2]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3)

In [5]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.5, nthread=7)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [6]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.8127340823970037

In [7]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [8]:
param = {'max_depth': 2, 'eta': 0.5, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 7
param['eval_metric'] = 'auc'
evallist = [(dtest, 'eval'), (dtrain, 'train')]
plst = param.items()

In [9]:
num_round = 50
bst = xgb.train(plst, dtrain, num_round, evallist)

[0]	eval-auc:0.842115	train-auc:0.845553
[1]	eval-auc:0.855109	train-auc:0.856217
[2]	eval-auc:0.859963	train-auc:0.863565
[3]	eval-auc:0.864018	train-auc:0.871598
[4]	eval-auc:0.862923	train-auc:0.870196
[5]	eval-auc:0.872543	train-auc:0.878065
[6]	eval-auc:0.878582	train-auc:0.884701
[7]	eval-auc:0.881216	train-auc:0.88881
[8]	eval-auc:0.869228	train-auc:0.893391
[9]	eval-auc:0.861858	train-auc:0.900532
[10]	eval-auc:0.86683	train-auc:0.902471
[11]	eval-auc:0.866801	train-auc:0.903118
[12]	eval-auc:0.869228	train-auc:0.904077
[13]	eval-auc:0.867511	train-auc:0.905989
[14]	eval-auc:0.869998	train-auc:0.90937
[15]	eval-auc:0.872602	train-auc:0.911327
[16]	eval-auc:0.872662	train-auc:0.912417
[17]	eval-auc:0.873431	train-auc:0.912801
[18]	eval-auc:0.874556	train-auc:0.914039
[19]	eval-auc:0.874023	train-auc:0.914823
[20]	eval-auc:0.878286	train-auc:0.915393
[21]	eval-auc:0.878789	train-auc:0.916801
[22]	eval-auc:0.879262	train-auc:0.917458
[23]	eval-auc:0.881423	train-auc:0.920565
[24]	

In [10]:
ypred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

In [11]:
(sum((ypred>0.5) == y_test)) / 267.0

0.8239700374531835

In [12]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np

In [13]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [15]:
import lightgbm as lgb

estimator = lgb.LGBMClassifier(num_leaves=2)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.05, 0.5, 1],
    'n_estimators': [20, 40, 60, 80, 100, 120]
}

gbm = GridSearchCV(estimator, param_grid, cv=5, scoring='roc_auc')

gbm.fit(X, y)

print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.5, 'n_estimators': 80}


In [16]:
sum(gbm.best_estimator_.predict(X_test) == y_test) / (len(y_test)*1.0)

0.8539325842696629

In [17]:
sum(
    (gbm.best_estimator_.predict_proba(X_test)[:,1]> 0.51 ) == y_test) / (len(y_test)*1.0)

0.8539325842696629

In [18]:

from xgboost.sklearn import XGBClassifier
estimator = XGBClassifier()

In [19]:
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}

In [20]:

clf = GridSearchCV(estimator, parameters, n_jobs=5, 
                   cv=5, 
                   scoring='roc_auc',
                   verbose=2, refit=True)

clf.fit(X, y)
print('Best parameters found by grid search are:', gbm.best_params_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:   10.8s remaining:   16.3s


Best parameters found by grid search are: {'learning_rate': 0.5, 'n_estimators': 80}


[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   11.3s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:   11.3s finished


In [21]:
sum(clf.best_estimator_.predict(X_test) == y_test) / (len(y_test)*1.0)

0.8052434456928839

In [22]:
sum(
    (clf.best_estimator_.predict_proba(X_test)[:,1]> 0.51 ) == y_test) / (len(y_test)*1.0)

0.8164794007490637